In [124]:
from BldgFinder import BldgFinder

## -------------------------------------------------------------------

In [127]:
test_addresses = ["1 New Fetter Lane London", "62 Mittelstraße Berlin 10117 DE", "21 Mittelstraße Berlin 10117 DE",
                  "164 Friedrichstraße Berlin", "18 Bloomsbury Square", "9 Bloomsbury Street", "374 Rue Saint-Honore Paris"]

In [126]:
mapper = BldgFinder('london')

Retrieved 20,458 bldgs in London


In [129]:
mapper.find(test_addresses[1])

Address not found.
Make sure the address (62 Mittelstraße Berlin 10117 DE) belongs to London, GB


SystemExit: 

## OLD code (pre-class)

In [1256]:
# TODO: restrict search by country


def get_closest_bldg(obj):
#     if len(res) > 1:
#         res = [obj for obj in res if obj['geojson']['type']=='Point'][0]
#     if len(res)==0:
#         return None
    gj = obj['geojson']
        
    if (gj['type'] == 'Point'):
        closest_bldg = gcomb[gcomb.geometry.intersects(Point(gj['coordinates']))]

        if not closest_bldg.empty:
            print("pin inside bldg")
            return closest_bldg
        else:
            print("pin outside, used closest bldg")
            closest_bldg = gcomb.loc[gcomb.geometry.distance(Point(gj['coordinates'])).sort_values(ascending=True)[:1].index, :]
            return closest_bldg
    else:
        print("poly: centroid to centroid closest bldg")
        closest_bldg = gcomb.loc[gcomb.centroid.distance(Point(float(obj['lon']), float(obj['lat']))).sort_values(ascending=True)[:1].index, :]
        return closest_bldg


    
def create_text_box(obj, dfdict):
    pophtml = """
        <h3> {title} </h3>
        <b>Baya ID:</b> {bid}<br>
        Country: {country}<br>
        """.format(title=', '.join(obj['display_name'].split(',')[:3]), 
                   bid=dfdict['id'], country=obj['address']['country'])
    for k,v in dfdict.items():
        if ('id' not in k) and ('country' not in k):
            pophtml = pophtml + "{}: {}<br>".format(k,v)
    return pophtml    
    

# TODO: unprint comment statemetns
def make_map(obj, closest_bldg, pophtml):
    # creates map
    bldg_poly = None
    gj = obj['geojson']
    address = obj['address']
    
    test_map = folium.Map(location=[float(obj['lat']), float(obj['lon'])], zoom_start=16)
    
#     if len(res) > 1:
#         res = [obj for obj in res if obj['geojson']['type']=='Point'][0]
#     if len(res)==0:
#         return None

    obj_point = folium.Marker(location = (float(obj['lat']), float(obj['lon'])), color='red')
    obj_point.add_to(test_map)
    

    if (gj['type'] == 'Point'):
        # creates a pin for the bldg
#         obj_point = folium.Marker(location = (gj['coordinates'][1], gj['coordinates'][0]), color='red')
#         obj_point.add_to(test_map)

        # delineates the surrounding/closest bldg polygon
        bldg_poly = folium.Polygon(locations = closest_bldg.geo_inv.values.tolist(),
                               color="red", fill=True, fill_color='#FF0000',
#                              address['house_number']+' '+address['road']+', '+address['suburb']
                               tooltip=folium.Tooltip(', '.join(obj['display_name'].split(',')[:3])),
                               popup=folium.Popup(create_text_box(obj, closest_bldg.iloc[:, :-3].drop('geo', axis=1).dropna(axis=1).to_dict(orient='rows')[0]), 
                                                     max_width=300))
        bldg_poly.add_to(test_map)
    
    if (gj['type'] == 'Polygon' and not bldg_poly):
        obj_poly = folium.Polygon(locations = [[ (a[1],a[0]) for a in b] for b in gj['coordinates']], 
                                  color="red", fill=True, fill_color='#FF0000',
                                  tooltip=folium.Tooltip(', '.join(obj['display_name'].split(',')[:3])),
                                  popup=folium.Popup(create_text_box(obj, closest_bldg.iloc[:, :-3].drop('geo', axis=1).dropna(axis=1).to_dict(orient='rows')[0]), 
                                                     max_width=300))
        obj_poly.add_to(test_map)
        
        
      
    display(test_map)



In [1257]:
test_addresses = ["1 New Fetter Lane London", "62 Mittelstraße Berlin 10117 DE", "21 Mittelstraße Berlin 10117 DE",
                  "164 Friedrichstraße Berlin", "18 Bloomsbury Square", "9 Bloomsbury Street", "374 Rue Saint-Honore Paris"]

In [1258]:
test_addresses[6]

'374 Rue Saint-Honore Paris'

In [1259]:
res

[]

In [1261]:
res = make_calls_to_Nominatim(test_addresses[6]) #test_addresses[3])
cb = get_closest_bldg(res[0])
make_map(res[0], cb, pophtml)

pin inside bldg
